# Multiple models comparison

This notebook will run three forecasting algorithms on the same dataset and compare their performances.

The algorithms are:
  - Prophet
  - ETS
  - DeepAR+
 

## Setup

In [ ]:
import sys
import os
import time
import pprint

import boto3
import pandas as pd

# importing forecast notebook utility from notebooks/common directory
sys.path.insert( 0, os.path.abspath("../../common") )
import util

pp = pprint.PrettyPrinter(indent=2)  # Better display for dictionaries

Configure the S3 bucket name and region name for this lesson.

- If you don't have an S3 bucket, create it first on S3.
- Although we have set the region to us-west-2 as a default value below, you can choose any of the regions that the service is available in.

In [ ]:
text_widget_bucket = util.create_text_widget( "bucket_name", "input your S3 bucket name" )
text_widget_region = util.create_text_widget( "region", "input region name.", default_value="us-west-2" )

In [ ]:
bucket_name = text_widget_bucket.value
assert bucket_name, "bucket_name not set."

region = text_widget_region.value
assert region, "region not set."

The last part of the setup process is to validate that your account can communicate with Amazon Forecast, the cell below does just that.

In [ ]:
session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

In [ ]:
# Create the role to provide to Amazon Forecast.
role_name = "ForecastNotebookRole-CompareMultipleModels"
role_arn = util.get_or_create_iam_role( role_name = role_name )

## Prepare your data

Preparing dataset with following steps.
1. Upload the CSV file to S3.
2. Create a DatasetGroup.
3. Create a Dataset and associate it with the DatasetGroup.
4. Import the uploaded CSV file to the Dataset.

#### Upload the CSV file to S3

In [ ]:
s3 = session.client('s3')
key="elec_data/item-demand-time.csv"
s3.upload_file(Filename="../../common/data/item-demand-time.csv", Bucket=bucket_name, Key=key)

In [ ]:
project = 'compare_multiple_models'
dataset_name= project+'_ds'
dataset_group_name= project +'_dsg'
s3_data_path = "s3://"+bucket_name+"/"+key

#### Create a DatasetGroup

In [ ]:
create_dataset_group_response = forecast.create_dataset_group(
    DatasetGroupName=dataset_group_name,
    Domain="CUSTOM",
    )

dataset_group_arn = create_dataset_group_response['DatasetGroupArn']

#### Create a Dataset and associate it with the DatasetGroup

In [ ]:
DATASET_FREQUENCY = "H"
TIMESTAMP_FORMAT = "yyyy-MM-dd hh:mm:ss"

schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"target_value",
         "AttributeType":"float"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      }
   ]
}

response=forecast.create_dataset(
    Domain="CUSTOM",
    DatasetType='TARGET_TIME_SERIES',
    DatasetName=dataset_name,
    DataFrequency=DATASET_FREQUENCY, 
    Schema = schema
)

dataset_arn = response['DatasetArn']

forecast.update_dataset_group(DatasetGroupArn=dataset_group_arn, DatasetArns=[dataset_arn])

#### Import the uploaded CSV file to the Dataset

In [ ]:
datasetImportJobName = 'EP_DSIMPORT_JOB_TARGET'
ds_import_job_response = forecast.create_dataset_import_job(
    DatasetImportJobName = datasetImportJobName,
    DatasetArn = dataset_arn,
    DataSource = {
        "S3Config" : {
            "Path":s3_data_path,
            "RoleArn": role_arn
        }
    },
    TimestampFormat=TIMESTAMP_FORMAT
)

ds_import_job_arn=ds_import_job_response['DatasetImportJobArn']

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

## Create the predictors

The next step is to create a dictionary where to store useful information about the algorithms: their name, ARN and eventually their performance metrics.

In [ ]:
algos = ['Prophet', 'ETS', 'Deep_AR_Plus']

predictors = {a:{} for a in algos}

for p in predictors:
    predictors[p]['predictor_name'] = project + '_' + p + '_algo'
    predictors[p]['algorithm_arn'] = 'arn:aws:forecast:::algorithm/' + p

pp.pprint(predictors)

Here we also define our forecast horizon: the number of time points to be predicted in the future. For weekly data, a value of 12 means 12 weeks. Our example is hourly data, we try forecast the next day, so we can set to 24.

In [ ]:
forecastHorizon = 24

The following function actually creates the predictor as specified by several parameters. We will call this function once for each of the 3 algorithms.

In [ ]:
def create_predictor_for_comparison(pred_name, algo_arn, forecast_horizon):
    response=forecast.create_predictor(PredictorName=pred_name, 
                                       AlgorithmArn=algo_arn,
                                       ForecastHorizon=forecast_horizon,
                                       PerformAutoML= False,
                                       PerformHPO=False,
                                       EvaluationParameters= {"NumberOfBacktestWindows": 1, 
                                                              "BackTestWindowOffset": 24}, 
                                       InputDataConfig= {"DatasetGroupArn": dataset_group_arn},
                                       FeaturizationConfig= {"ForecastFrequency": "H", 
                                                             "Featurizations": 
                                                             [
                                                                 {"AttributeName": "target_value", 
                                                                  "FeaturizationPipeline": 
                                                                  [
                                                                      {"FeaturizationMethodName": "filling", 
                                                                       "FeaturizationMethodParameters": 
                                                                       {"frontfill": "none", 
                                                                        "middlefill": "zero", 
                                                                        "backfill": "zero"}
                                                                      }
                                                                  ]
                                                                 }
                                                             ]
                                                            }
                                      )
    return response

For all 3 algorithms, we invoke their creation and wait until they are complete. We also store their performance in our dictionary.

In [ ]:
for p in predictors.keys():

    print('Creating predictor :', p)
    
    predictor_response = create_predictor_for_comparison(predictors[p]['predictor_name'], predictors[p]['algorithm_arn'], forecastHorizon)
    predictorArn=predictor_response['PredictorArn']
    
    # wait for the predictor to be actually created
    status_indicator = util.StatusIndicator()
    while True:
        status = forecast.describe_predictor(PredictorArn=predictorArn)['Status']
        status_indicator.update(status)
        if status in ('ACTIVE', 'CREATE_FAILED'): break
        time.sleep(10)
    status_indicator.end()            

    predictors[p]['predictor_arn'] = predictorArn  # save it, just for reference

    # compute and store performance metrics, then proceed with the next algorithm        
    predictors[p]['accuracy'] = forecast.get_accuracy_metrics(PredictorArn=predictorArn)

**TODO:** (Bar?)plot RMSE, 0.9-, 0.5- and 0.1-quantile LossValues for each algorithm

This is what we stored so far for DeepAR+:

In [ ]:
predictors['Deep_AR_Plus']

## Visualize results

Looping over our dictionary, we can retrieve the Root Mean Square Error (RMSE) for each predictor and plot it as a bar plot.

In [ ]:
scores = pd.DataFrame(columns=['predictor', 'RMSE'])
for p in predictors:
    score = predictors[p]['accuracy']['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['RMSE']
    scores = scores.append(pd.DataFrame({'predictor':[p], 'RMSE':[score]}), ignore_index=True)

scores.plot.bar( x="predictor", y="RMSE" )

## Cleanup
Deleting all Amazon Forecast resources we created above.

In [ ]:
# Delete predictors
for p in predictors:
    util.wait_till_delete(lambda: forecast.delete_predictor(PredictorArn = predictors[p]['predictor_arn']))

In [ ]:
# Delete dataset import job
util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=ds_import_job_arn))

In [ ]:
# Delete dataset
util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=dataset_arn))

In [ ]:
# Delete dataset group
forecast.delete_dataset_group(DatasetGroupArn=dataset_group_arn)

In [ ]:
# Delete IAM role
util.delete_iam_role( role_name )